In [0]:
# https://colab.research.google.com/notebooks/welcome.ipynb
# Google Colabのアカウント作って登録する
# FileをクリックしてNew python3 notebookを選ぶ
# New　Python3notebook　クリックする

# GPUで動き設定
# runtime(メニューのボタン)　クリックして change runtime type　クリックして　GPUに設定する

# google　colabからマウントする
from google.colab import drive
drive.mount('/content/gdrive')
# authorization code取得して入力する（enterを押すとマウント完了）

# 出力メッセージ:
# Enter your authorization code:
# ··········
# Mounted at /content/gdrive

In [0]:
# gitからプロジェクトをクローンする
! git clone https://github.com/PasonaTech-Inc/Qumico.git "/content/gdrive/My Drive/Qumico"
# 一度クローンした場合は　
# 'fatal: destination path '/content/gdrive/My Drive/Qumico' already exists and is not an empty directory.'って表示する

In [0]:
# pythonpathの環境変数設定
import os                                                                                                             
os.environ['PYTHONPATH'] += ":/content/gdrive/My Drive/Qumico"

In [0]:
# pythonpath設定された
! echo $PYTHONPATH
# 出力メッセージ:　/env/python:/content/gdrive/My Drive/Qumico

In [0]:
# ディレクトリを移動する
%cd /content/gdrive/My Drive/Qumico
# ディレクトリ確認する。
# 出力メッセージ: /content/gdrive/My Drive/Qumico

In [0]:
! pwd

In [0]:
# 必要なライブラリをインストールする
! pip install -r requirements.txt
! pip install tensorflow-gpu==1.13.2 

In [0]:
#path変わったため、writefileで実行する
%%writefile samples/tiny_yolo_v2/tensorflow/tiny_yolo2_train.py

import tensorflow as tf
import numpy as np
from samples.tiny_yolo_v2.tensorflow.tiny_yolo2_model import TINY_YOLO_v2
import samples.utils.pre_process_tool as list_reader
from samples.utils.annotation_dataset_tool import AnnotationDatasetTool
from samples.utils.data_augument_tool import (Transform, Sequence,RandomHorizontalFlip,RandomScale,RandomRotate,
                                         RandomTranslate, RandomHSV, RandomShear)
from qumico.Qumico import Qumico
#%% 変数変更どころ%%
import os


def retrain(model, train_data, ckpt_file, epochs, batch_size, save_flag=True):
    tf.reset_default_graph()

    pb_file = None
    data_size = train_data.total_size
    total_batch = data_size // batch_size

    with tf.Session(graph=model.graph) as sess_train:
        model.saver.restore(sess_train, ckpt_file)

        for epoch in range(epochs):
            total_loss = 0
            total_class_loss = 0
            total_obj_loss = 0
            total_coord_loss = 0
            for i in range(total_batch):
                batch_x, batch_y, _, _ = train_data.next_batch_once(batch_size=batch_size)
                y_true, y_mask, y_true_no_grid = model.pre_process_true_boxes_normal(batch_y, grid_w=model.grid_w, grid_h=model.grid_h)
                

                _, output, loss, class_loss, obj_loss, coord_loss = sess_train.run(
                    [model.train_op, model.output, model.total_loss, model.class_loss, model.object_loss, model.coord_loss],
                    feed_dict={model.inputs: batch_x, model.y_true: y_true, model.y_mask: y_mask, model.y_true_nogrid: y_true_no_grid})



                total_loss += loss
                total_class_loss += class_loss
                total_obj_loss += obj_loss
                total_coord_loss += coord_loss
                print("=" * 3, "epoch:", epoch + 1, " - batch", i + 1, "/", total_batch, "=" * 3)

            print("total loss : ", total_loss / data_size, " class loss : ", total_class_loss / data_size,
                  " obj loss : ", total_obj_loss / data_size, " coord_loss : ", total_coord_loss / data_size)


        if save_flag:
            ckpt_file = "model/tiny_yolo2.ckpt"
            pb_path = "model"
            pb_name = "tiny_yolo2.pb"
            pb_file = pb_path + "/" + pb_name
            model.saver.save(sess_train, ckpt_file)
            model.pb_saver.write_graph(sess_train.graph, pb_path, pb_name, as_text=False)

    return ckpt_file, pb_file

def train(model, train_data, epochs, batch_size, save_flag=True):
    tf.reset_default_graph()
    ckpt_file = None
    pb_file = None
    data_size = train_data.total_size
    total_batch = data_size // batch_size

    with tf.Session(graph=model.graph) as sess_train:
        sess_train.run(model.init)
        for epoch in range(epochs):
            total_loss = 0
            total_class_loss = 0
            total_obj_loss = 0
            total_coord_loss = 0
            for i in range(total_batch):
                batch_x, batch_y, _, _ = train_data.next_batch_once(batch_size=batch_size)
                y_true, y_mask, y_true_no_grid = model.pre_process_true_boxes_normal(batch_y, grid_w=model.grid_w, grid_h=model.grid_h)

                _, output, loss, class_loss, obj_loss, coord_loss = sess_train.run(
                    [model.train_op, model.output, model.total_loss, model.class_loss, model.object_loss, model.coord_loss],
                    feed_dict={model.inputs: batch_x, model.y_true: y_true, model.y_mask: y_mask, model.y_true_nogrid: y_true_no_grid})



                total_loss += loss
                total_class_loss += class_loss
                total_obj_loss += obj_loss
                total_coord_loss += coord_loss
                print("=" * 3, "epoch:", epoch + 1, " - batch", i + 1, "/", total_batch, "=" * 3)

            print("total loss : ", total_loss / data_size, " class loss : ", total_class_loss / data_size,
                  " obj loss : ", total_obj_loss / data_size, " coord_loss : ", total_coord_loss / data_size)


        if save_flag:
            ckpt_file = "model/tiny_yolo2.ckpt"
            pb_path = "model"
            pb_name = "tiny_yolo2.pb"
            pb_file = pb_path + "/" + pb_name
            model.saver.save(sess_train, ckpt_file)
            model.pb_saver.write_graph(sess_train.graph, pb_path, pb_name, as_text=False)

    return ckpt_file, pb_file


if __name__ == '__main__':

    # クラス　ラベル
    voc2007_classes = ['chair', 'bird', 'sofa', 'bicycle', 'cat', 'motorbike', 'bus', 'boat', 'sheep', 'bottle', 'cow',
                       'person', 'horse', 'diningtable', 'pottedplant', 'aeroplane', 'car', 'train', 'dog', 'tvmonitor']
    # クラスの数　Ont-Hot表現用パラメータ
    num_classes = len(voc2007_classes)

    #%% 変数変更どころ%%
    root_path = "/content/gdrive/My Drive/Qumico/samples/tiny_yolo_v2/tensorflow"       
    data_path = os.path.join(root_path, "data")

    #%% 変数変更どころ%%
    data_list_path = os.path.join(data_path, "JPEGImages")
    label_list_path = os.path.join(data_path, "Annotations")

    data_list = np.asarray(list_reader.get_data_path_list(data_list_path)[:])
    label_list = np.asarray(list_reader.get_data_path_list(label_list_path)[:])

    # Data Augument
    transformer = Sequence([RandomHorizontalFlip(), RandomScale(0.3, diff=True),
                            RandomHSV(hue=50, saturation=50, brightness=50), RandomShear(),RandomTranslate(diff=True)])

    # 読み方、バッチ処理などを設定し、データを供給するツール設定する
    annotation_dataset_tool = AnnotationDatasetTool(training_flag=True, data_list=data_list, label_list=label_list,
                                                    category_class=voc2007_classes, one_hot_classes=num_classes,
                                                    resize_flag=True, target_h=416, target_w=416,
                                                    label_file_type="voc_xml", format="NCHW", data_rescale=True, label_resclar=True,
                                                    transformer=transformer)

    # バッチサイズ
    batch_size = 1
    epoch_num = 2
    total_size = annotation_dataset_tool.total_size

    range_size = int(total_size / batch_size)
    # load model
    tiny_yolo_2 = TINY_YOLO_v2(height=416, width=416, output_op_name="output", num_classes=20, is_train=True,
                               batch_size=batch_size)

    # train and save ckpt pb file
    # ====train==================================================================================================
    ckpt_file, pb_file = train(tiny_yolo_2, annotation_dataset_tool, epoch_num, batch_size, save_flag=True)
    # ====retain============================================================================================
    #%% 変数変更どころ%%
    ckpt_file = os.path.join(root_path, "model/tiny_yolo2.ckpt")
    # pb_file= "model/tiny_yolo2.pb"     
    #ckpt_file, pb_file = retrain(tiny_yolo_2, annotation_dataset_tool, ckpt_file, epoch_num, batch_size, save_flag=True)
    # ======================================================================================================

    print("ckpt_file", ckpt_file)
    print("pb_file", pb_file)

    # prepare Qumico Convertor
    converter = Qumico()
    converter.conv_tf_to_onnx(output_path="onnx", model_name="tiny_yolo_2", output_op_name="output",
                              cache_path="model", ckpt_file=ckpt_file, pb_file=pb_file)

In [0]:
# flatcの実行権限与える
!chmod +x "/content/gdrive/My Drive/Qumico/external/flatbuffers/flatc"
!ls -la "/content/gdrive/My Drive/Qumico/external/flatbuffers/flatc"
# 出力メッセージ:　rwx------ 1 root root 3771528 Dec 23 01:27 '/content/gdrive/My Drive/Qumico/external/flatbuffers/flatc'

In [0]:
# tiny_yolo_v2のonnxファイル作る
! python samples/tiny_yolo_v2/tensorflow/tiny_yolo2_train.py
# 出力メッセージ:　 
# onnx/tiny_yolo_2.onnxを作成しました。

In [0]:
# out_cのフォルダ作る(デフォルトはFalse)
! python samples/tiny_yolo_v2/tensorflow/gen_c.py
# 出力メッセージ:  　
# Cソースを生成しました。出力先: out_c

In [0]:
#一旦qumico.so削除する
! rm samples/tiny_yolo_v2/tensorflow/out_c/qumico.so
#削除チェック
! ls samples/tiny_yolo_v2/tensorflow/out_c

In [0]:
#known_issues フォルダ　の何にあるbatchnormalization.cとmaxpool.c
#二つのファクタをout_cフォルダのlibにコピーしてまたqumico.cをcompileする
! gcc -c -fPIC samples/tiny_yolo_v2/tensorflow/out_c/qumico.c -o file.o
! gcc file.o -shared -o samples/tiny_yolo_v2/tensorflow/out_c/qumico.so
#qumico.so チェック
! ls samples/tiny_yolo_v2/tensorflow/out_c

In [0]:
# ministデータ十個取得、　トレーニングしたモデルで推論した結果引き出す
# (notebookでopen_cvから画像出力できないため、代わりにipythonで出力することになります)
import os
from google.colab.patches import cv2_imshow
import numpy as np
from samples.utils.annotation_dataset_tool import AnnotationDatasetTool
from samples.tiny_yolo_v2.tensorflow.tiny_yolo2_infer import infer
from samples.tiny_yolo_v2.tensorflow import tiny_yolo2_infer
import samples.utils.pre_process_tool as list_reader
from samples.tiny_yolo_v2.tensorflow.tiny_yolo2_model import TINY_YOLO_v2

voc2007_classes = ['chair', 'bird', 'sofa', 'bicycle', 'cat', 'motorbike', 'bus', 'boat', 'sheep', 'bottle', 'cow',
                       'person', 'horse', 'diningtable', 'pottedplant', 'aeroplane', 'car', 'train', 'dog', 'tvmonitor']

num_classes = len(voc2007_classes)
root_path = "/content/gdrive/My Drive/Qumico/samples/tiny_yolo_v2/tensorflow"
data_path = os.path.join(root_path, "data")
data_list_path = os.path.join(data_path, "JPEGImages")
label_list_path = os.path.join(data_path, "Annotations")
pic_num = 0
data_list = np.asarray(list_reader.get_data_path_list(data_list_path)[pic_num:pic_num+1])
label_list = np.asarray(list_reader.get_data_path_list(label_list_path)[pic_num:pic_num+1])
annotation_dataset_tool = AnnotationDatasetTool(training_flag=True, data_list=data_list, label_list=label_list,
                                                    category_class=voc2007_classes, one_hot_classes=num_classes,
                                                    resize_flag=True, target_h=416, target_w=416,
                                                    label_file_type="voc_xml", format="NCHW", data_rescale=True, label_resclar=True)
batch_size = 1
tiny_yolo2_model = TINY_YOLO_v2(output_op_name="output", num_classes=20, is_train=False, width=416, height=416)
ckpt_file = os.path.join(root_path, "model/tiny_yolo2_backup.ckpt")
print(ckpt_file)
res = tiny_yolo2_infer.infer(tiny_yolo2_model, annotation_dataset_tool, ckpt_file, voc2007_classes, batch_size, to_draw=False)
print(res)
 

from PIL import Image
import numpy as np
from IPython.display import display
w, h = 300, 300
data = np.zeros((h, w, 3), dtype=np.uint8)
data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(res)
display(img)

# Sunflower

In [0]:
# ministデータ十個取得、　トレーニングしたモデルで推論した結果引き出す
# (notebookでopen_cvから画像出力できないため、代わりにipythonで出力することになります)
import os
from google.colab.patches import cv2_imshow
import numpy as np
from samples.utils.annotation_dataset_tool import AnnotationDatasetTool
from samples.tiny_yolo_v2.tensorflow.tiny_yolo2_infer_c import infer_c
from samples.tiny_yolo_v2.tensorflow import tiny_yolo2_infer_c
import samples.utils.pre_process_tool as list_reader
from samples.tiny_yolo_v2.tensorflow.tiny_yolo2_model import TINY_YOLO_v2

voc2007_classes = ['chair', 'bird', 'sofa', 'bicycle', 'cat', 'motorbike', 'bus', 'boat', 'sheep', 'bottle', 'cow',
                       'person', 'horse', 'diningtable', 'pottedplant', 'aeroplane', 'car', 'train', 'dog', 'tvmonitor']

                       
num_classes = len(voc2007_classes)
root_path = '/content/gdrive/My Drive/Qumico/samples/tiny_yolo_v2/tensorflow'
c_path = os.path.join(root_path, "out_c", "qumico.so") 
data_path = os.path.join(root_path, "data")
data_list_path = os.path.join(data_path, "JPEGImages")
label_list_path = os.path.join(data_path, "Annotations")
pic_num = 0
data_list = np.asarray(list_reader.get_data_path_list(data_list_path)[pic_num:pic_num+1])
label_list = np.asarray(list_reader.get_data_path_list(label_list_path)[pic_num:pic_num+1])
annotation_dataset_tool = AnnotationDatasetTool(training_flag=True, data_list=data_list, label_list=label_list,
                                                    category_class=voc2007_classes, one_hot_classes=num_classes,
                                                    resize_flag=True, target_h=416, target_w=416,
                                                    label_file_type="voc_xml", format="NCHW", data_rescale=True, label_resclar=True)
batch_size = 1
tiny_yolo2_model = TINY_YOLO_v2(output_op_name="output", num_classes=20, is_train=False, width=416, height=416)
res = tiny_yolo2_infer_c.infer_c(tiny_yolo2_model, annotation_dataset_tool, c_path, voc2007_classes, batch_size, to_draw=True)
print(res)
 

from PIL import Image
import numpy as np
from IPython.display import display
w, h = 300, 300
data = np.zeros((h, w, 3), dtype=np.uint8)
data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(res)
display(img)

# Sunflower

In [0]:
# ディレクトリ移します。
%cd samples/tiny_yolo_v2/tensorflow/
# ディレクトリ確認します。
# 出力メッセージ /content/gdrive/My Drive/Qumico/samples/tiny_yolo_v2/tensorflow/

In [0]:
# out_cフォルダ存在するがどうか確認する
! ls 
# data	     model  __pycache__  tiny_yolo2_infer_c.py	tiny_yolo2_train.py
# gen_c.py     onnx   testfile	 tiny_yolo2_infer.py
# known_issue  out_c  testnum.py	 tiny_yolo2_model.py

In [0]:
# qumico.so存在するがどうか確認する
! ls out_c
# 出力メッセージ:
# include  initializers  lib  numpy.c  qumico.c  qumico.so

In [0]:
# out_c file zipして出力する 
!zip -r '/content/gdrive/My Drive/out_c.zip' out_c  